In [8]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')
# from zigzag import *
from tqdm import tqdm
import sys

In [3]:

data = pd.read_csv(f'C:\\Users\\HFY\\Desktop\\capstone\\data\\Data_OANDA\\AUD_USD_M15_2005_202206.csv', encoding='utf-8',header=0)
#data['datetime'] = pd.to_datetime(data['datetime'], format='%Y-%m-%dT%H:%M:%S.000000000Z')
data.reset_index(drop=True, inplace=True)

In [9]:
# Buy Hold Sell with Window Size

def bhs_ws_label(data,window_size,pt,sl,currency_pair_file):
    file_name = f'ws{window_size}_pt{pt}_sl{sl}'
    data.fillna('',inplace = True)
    data[file_name] = ''
    ONE_PIP = 0.0001
    stop_loss = ONE_PIP * sl
    profit_taken = ONE_PIP * pt
    action_ls = []

    for row in tqdm(range(0,len(data) -1)):

        current_open,current_close,current_low,current_high = data['open'][row], data['close'][row], data['low'][row], data['high'][row]

        action_ls.append('hold')
        if data[file_name][row] == '':

            enter_price =  current_close
    
            max_given_ws = data['high'].iloc[row+1 : row+window_size+1].max()
            min_given_ws = data['low'].iloc[row+1: row+window_size+1].min()

            max_seq = data['high'].iloc[row+1 : row+window_size+1]
            min_seq =  data['low'].iloc[row+1: row+window_size+1]
            
            max_index = max_seq[max_seq==max_given_ws].index[0]
            min_index = min_seq[min_seq==min_given_ws].index[0]
            
            if (max_given_ws - enter_price)/ONE_PIP >= pt and (enter_price - min_given_ws)/ONE_PIP<sl:
                action_ls.pop()
                action_ls.append('buy')

            elif (max_given_ws - enter_price)/ONE_PIP >= pt and (enter_price - min_given_ws)/ONE_PIP>=sl:
                if max_index<= min_index:
                    action_ls.pop()
                    action_ls.append('buy')
                else:
                    action_ls.pop()
                    action_ls.append('hold')

            elif (enter_price - min_given_ws)/ONE_PIP>= pt and (max_given_ws - enter_price)/ONE_PIP<sl:
                action_ls.pop()
                action_ls.append('sell')
            
            elif (enter_price - min_given_ws)/ONE_PIP>= pt and (max_given_ws - enter_price)/ONE_PIP>=sl:
                if min_index<= max_index:
                    action_ls.pop()
                    action_ls.append('sell')
                else:
                    action_ls.pop()
                    action_ls.append('hold')


    action_ls.append('hold')
    data[file_name] = action_ls
    # data.to_csv(f'C:\\Users\\HFY\\Desktop\\capstone\\w11\\label\\{currency_pair_file}_{file_name}.csv',index =False)
           
           
# No Window Size
def bhs_no_ws_label(data,pt,sl,currency_pair_file):
    file_name = f'nows_pt{pt}_sl{sl}'
    data.fillna('',inplace = True)
    data[file_name] = ''
    ONE_PIP = 0.0001
    stop_loss = ONE_PIP * sl
    profit_taken = ONE_PIP * pt
    action_ls = []

    for row in tqdm(range(0,len(data) -1)):
        not_found = True
        current_open,current_close,current_low,current_high = data['open'][row], data['close'][row], data['low'][row], data['high'][row]

        action_ls.append('hold')
        if data[file_name][row] == '' and row<=len(data):
            index = 1
            enter_price =  current_close

            while not_found:
                max_given_ws = data['high'].iloc[row+1 : row+1+index].max()
                min_given_ws = data['low'].iloc[row+1 : row+1+index].min()

                max_seq = data['high'].iloc[row+1 : row+1+index]
                min_seq =  data['low'].iloc[row+1 : row+1+index]
                #print(max_seq)
                max_index = max_seq[max_seq==max_given_ws].index[0]
                min_index = min_seq[min_seq==min_given_ws].index[0]
                index+=1

                if (max_given_ws - enter_price)/ONE_PIP >= pt and (enter_price - min_given_ws)/ONE_PIP<sl:
                    action_ls.pop()
                    action_ls.append('buy')
                    not_found = False

                elif (max_given_ws - enter_price)/ONE_PIP >= pt and (enter_price - min_given_ws)/ONE_PIP>=sl:
                    if min_index<=max_index:
                        action_ls.pop()
                        action_ls.append('hold')
                        not_found = False
                    else:
                        action_ls.pop()
                        action_ls.append('buy')
                        not_found = False

                elif (enter_price - min_given_ws)/ONE_PIP>= pt and (max_given_ws - enter_price)/ONE_PIP<sl:
                    action_ls.pop()
                    action_ls.append('sell')
                    not_found = False

                elif (enter_price - min_given_ws)/ONE_PIP>= pt and (max_given_ws - enter_price)/ONE_PIP>=sl:
                    if max_index<=min_index:
                        action_ls.pop()
                        action_ls.append('hold')
                        not_found = False
                    else:
                        action_ls.pop()
                        action_ls.append('sell')
                        not_found = False
                not_found = False


    action_ls.append('hold')
    data[file_name] = action_ls
    # data.to_csv(f'C:\\Users\\HFY\\Desktop\\capstone\\w11\\label\\{currency_pair_file}_{file_name}.csv',index =False)
           

In [ ]:
data.action.value_counts()

In [10]:
FILE_NAME_LIST = ['AUD_CAD_M15_2005_202206'
]

#change directroy
data_ls = []
for i in FILE_NAME_LIST:
    file = pd.read_csv(f'C:\\Users\\HFY\\Desktop\\capstone\\data\\Data_OANDA\\{i}.csv')
    file.reset_index(drop=True, inplace=True)
    data_ls.append(file)


In [12]:
for i in range(len(data_ls)):
    # with window size

    bhs_ws_label(data_ls[i],10,8,15,FILE_NAME_LIST[i])
    bhs_ws_label(data_ls[i],15,10,20,FILE_NAME_LIST[i])
    bhs_ws_label(data_ls[i],20,10,30,FILE_NAME_LIST[i])
    bhs_ws_label(data_ls[i],30,20,40,FILE_NAME_LIST[i])
    bhs_ws_label(data_ls[i],40,20,40,FILE_NAME_LIST[i])

    bhs_ws_label(data_ls[i],10,10,8,FILE_NAME_LIST[i])
    bhs_ws_label(data_ls[i],15,15,20,FILE_NAME_LIST[i])
    bhs_ws_label(data_ls[i],20,20,18,FILE_NAME_LIST[i])
    bhs_ws_label(data_ls[i],30,30,28,FILE_NAME_LIST[i])
    bhs_ws_label(data_ls[i],40,30,28,FILE_NAME_LIST[i])
    
    # no window size

    bhs_no_ws_label(data_ls[i],8,15,FILE_NAME_LIST[i])
    bhs_no_ws_label(data_ls[i],10,20,FILE_NAME_LIST[i])
    bhs_no_ws_label(data_ls[i],10,30,FILE_NAME_LIST[i])
    bhs_no_ws_label(data_ls[i],20,40,FILE_NAME_LIST[i])


    bhs_no_ws_label(data_ls[i],10,8,FILE_NAME_LIST[i])
    bhs_no_ws_label(data_ls[i],15,12,FILE_NAME_LIST[i])
    bhs_no_ws_label(data_ls[i],20,18,FILE_NAME_LIST[i])
    bhs_no_ws_label(data_ls[i],30,28,FILE_NAME_LIST[i])
    data_ls[i].to_csv(f'C:\\Users\\HFY\\Desktop\\capstone\\w11\\label\\{FILE_NAME_LIST[i]}_labels_eurusd_nows.csv',index =False)



100%|██████████| 440867/440867 [04:57<00:00, 1480.94it/s]
